In [1]:
import photo_preparation_functions as ppf
import dataset_generation_functions as dgf
import os

In [2]:
PROJECT_DIR = '/home/moiki/Documents/Files/studies/4_Semester/ADL/ADL_W23'
relative_location_videos = '/data/sg_cards'
relative_path_backgrounds = '/data/dtd'
relative_path_dataset = '/data/sg_cards_dataset_1'

n_frames = 10
sensitivity = 8
testing = False

seed = 1
number_of_images = 2
max_number_of_cards = 3
overlapping = False
max_size = 0.4
min_size = 0.1
max_rotation = 90
max_angle = 45
#color transformations,...?

In [3]:
PLAYING_CARDS_DIR = str(PROJECT_DIR) + relative_location_videos
BACKGROUNDS_DIR = str(PROJECT_DIR) + relative_path_backgrounds
GENERATED_DATASET_DIR = str(PROJECT_DIR) + relative_path_dataset


PLAYING_CARDS_FRAMES_DIR = ppf.process_videos(PLAYING_CARDS_DIR, n_frames, sensitivity, testing)
PROCESSED_PLAYING_CARDS_DIR = ppf.process_photos(PLAYING_CARDS_FRAMES_DIR)
print(f'Processed playing cards saved at {PROCESSED_PLAYING_CARDS_DIR}!')

dgf.generate_dataset(BACKGROUNDS_DIR, PLAYING_CARDS_DIR, PROCESSED_PLAYING_CARDS_DIR, GENERATED_DATASET_DIR, number_of_images, max_number_of_cards, 
                                             min_size, max_size, max_rotation, max_angle, overlapping, seed)
print(f'Generated Dataset saved at {GENERATED_DATASET_DIR}!')

Processed playing cards saved at /home/moiki/Documents/Files/studies/4_Semester/ADL/ADL_W23/data/sg_cards_frames_processed!
Generated Dataset saved at /home/moiki/Documents/Files/studies/4_Semester/ADL/ADL_W23/data/sg_cards_dataset_1!
